In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Thymoma/GSE29695'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Analysis of Thymic Neoplasms"
!Series_summary	"Histologic classification of thymomas has significant limitations since complete surgical excision can be curative. In order to better understand the biology of the disease processes, we performed whole genome gene expression analysis. RNA was extracted from fresh frozen tumors from 36 patients with thymomas and follow-up data was available. Gene expression data was correlated with clinicopathological data. Using the Illumina BeadStudio® platform and Human Ref-8 Beadchip, gene expression data was analyzed by Partek®, and Ingenuity Pathways Analysis (IPA). Validation of the chosen genes was performed using quantitative real-time RT-PCR (qRT-PCR). Unsupervised clustering resulted in identification of four clusters of tumors (C1-C4). Using IPA, the top significant biological functions and pathways displayed cell cycle related category and genes in C1 and C2. Carbohydrate metabolism and ce

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True

sample_characteristics = {
    0: ['tissue: Fresh Frozen Human Tumors', 'tissue: Cell Line'],
    1: ['type: B1', 'type: Mixed AB', 'type: CL', 'type: B2', 'type: B3', 'type: AB', 'type: A/B', 'type: B1/B2', 'type: A'],
    2: ['category: GII', 'category: GI', 'category: CL', 'category: GIII'],
    3: ['batch group: BATCH 1', 'batch group: BATCH 2', 'batch group: BATCH 3'],
    4: ['stage i/ii, iii/iv, or na = not applicable/unknown: III_IV', 'stage i/ii, iii/iv, or na = not applicable/unknown: NA', 'stage i/ii, iii/iv, or na = not applicable/unknown: I_II'],
    5: ['relapse no, yes, or na = not applicable/unknown: NA', 'relapse no, yes, or na = not applicable/unknown: NO', 'relapse no, yes, or na = not applicable/unknown: YES'],
    6: ['metastasis no, yes, or na = not applicable/unknown: NA', 'metastasis no, yes, or na = not applicable/unknown: YES', 'metastasis no, yes, or na = not applicable/unknown: NO']
}

trait_row = 1
age_row = None
gender_row = None

def convert_trait(value):
    trait_mapping = {
        'B1': 0,
        'Mixed AB': 1,
        'CL': None,  # ignoring cell lines
        'B2': 2,
        'B3': 3,
        'AB': 4,
        'A/B': 5,
        'B1/B2': 6,
        'A': 7
    }
    val = value.split(": ")[1]
    result = trait_mapping.get(val, None)
    print(f"Converting trait: {value} -> {result}")  # Debug output
    return result

def convert_age(value):
    return None

def convert_gender(value):
    return None

save_cohort_info('GSE29695', './preprocessed/Thymoma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    print("Clinical data before extraction:")
    print(clinical_data.head())
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Thymoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    print("Clinical data after extraction:")
    print(selected_clinical_data.head())
    
    csv_path = './preprocessed/Thymoma/trait_data/GSE29695.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


Clinical data before extraction:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   
3  !Sample_characteristics_ch1   
4  !Sample_characteristics_ch1   

                                           GSM736670  \
0                  tissue: Fresh Frozen Human Tumors   
1                                           type: B1   
2                                      category: GII   
3                               batch group: BATCH 1   
4  stage i/ii, iii/iv, or na = not applicable/unk...   

                                           GSM736671  \
0                  tissue: Fresh Frozen Human Tumors   
1                                           type: B1   
2                                      category: GII   
3                               batch group: BATCH 1   
4  stage i/ii, iii/iv, or na = not applicable/unk...   

                                           GSM736672  \
0                  tissue: Fresh

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_2161508', 'ILMN_1796063', 'ILMN_1668162', 'ILMN_1793729', 'ILMN_2296644'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_13666', 'ILMN_5006', 'ILMN_7652', 'ILMN_18382', 'ILMN_22537'], 'Transcript': ['ILMN_13666', 'ILMN_5006', 'ILMN_7652', 'ILMN_18382', 'ILMN_22537'], 'ILMN_Gene': ['PHTF2', 'TRIM44', 'DGAT2L3', 'C15ORF39', 'PCDHGA9'], 'Source_Reference_ID': ['NM_020432.2', 'NM_017583.3', 'NM_001013579.1', 'NM_015492.4', 'NM_018921.2'], 'RefSeq_ID': ['NM_020432.2', 'NM_017583.3', 'NM_001013579.1', 'NM_015492.4', 'NM_018921.2'], 'Entrez_Gene_ID': [57157.0, 54765.0, 158833.0, 56905.0, 56107.0], 'GI': [40254932.0, 29029528.0, 61888901.0, 153251858.0, 14270485.0], 'Accession': ['NM_020432.2', 'NM_017583.3', 'NM_001013579.1', 'NM_015492.4', 'NM_018921.2'], 'Symbol': ['PHTF2', 'TRIM44', 'DGAT2L3', 'C15orf39', 'PCDHGA9'], 'Prote

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Thymoma/gene_data/GSE29695.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Thymoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE29695', './preprocessed/Thymoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased and unbiased_merged_data is not None:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Thymoma/GSE29695.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Thymoma':
  25%: 0.0
  50% (Median): 2.0
  75%: 3.0
Min: 0.0
Max: 7.0
The distribution of the feature 'Thymoma' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Thymoma/cohort_info.json
